In [1]:
#another subset of random edges removed - to see if the results are consistent

In [2]:
import numpy as np
import pickle as pkl
import torch
from scipy.spatial.distance import cosine, euclidean
from scipy import stats
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [3]:
#import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
import networkx as nx

In [6]:
import operator
import os

In [7]:
%matplotlib inline

In [8]:
import bz2
import _pickle as cPickle

# Load any compressed pickle file
def decompress_pickle(file):
    assert file.endswith("pbz2")
    data = bz2.BZ2File(file, "rb")
    data = cPickle.load(data,encoding="latin1")
    return data

In [9]:
#data

In [10]:
version_no = "2"

In [11]:
dname = "./../../data_hin/PubMed_orig/"
data_dir = "./../../data/pubmed_heuristic/"
out_dir = "./../../out_clust/pubmed_heuristic/version_"+version_no+"/cc/"

#
out_dir_min_max = out_dir+"PubMed_da_link/"
#out_dir_min = dname+"filter_min_edges/"
# out_dir_rand1 = dname_mat+"filter_rand1_max_edges/" #matches max edge removal count - remove from only the matrices in the CC
# out_dir_rand2 = dname_mat+"filter_rand2_const_edges/" #fixed num of edges removal - remove from only the matrices in the CC
# out_dir_rand3 = dname_mat+"filter_rand3_min_edges/" #matches min edge removal count - remove from only the matrices in the CC
#
# out_dir_rand4 = dname_mat+"filter_rand4_max_edges_all_mat/" #matches max edge removal count, but remove from all the 10 matrices randomly and NOT remove from only the matrices in the CC
# out_dir_rand5 = dname_mat+"filter_rand5_min_edges_all_mat/"
#
out_dir_rand8 = out_dir+"PubMed_rand_link/" #same as prev two, but distibte the edges count to all matrices uniformly
#out_dir_rand7 = dname+"filter_rand7_min_uniform/"

In [12]:
if not os.path.exists(out_dir_min_max):
    os.makedirs(out_dir_min_max)
#
# if not os.path.exists(out_dir_max):
#     os.makedirs(out_dir_max)
# #
# if not os.path.exists(out_dir_min):
#     os.makedirs(out_dir_min)
#
# if not os.path.exists(out_dir_rand1):
#     os.makedirs(out_dir_rand1)
# #
# if not os.path.exists(out_dir_rand2):
#     os.makedirs(out_dir_rand2)
# #
# if not os.path.exists(out_dir_rand3):
#     os.makedirs(out_dir_rand3)  

In [13]:
# #
# if not os.path.exists(out_dir_rand4):
#     os.makedirs(out_dir_rand4)
# #
# if not os.path.exists(out_dir_rand5):
#     os.makedirs(out_dir_rand5)
# #
# if not os.path.exists(out_dir_rand6):
#     os.makedirs(out_dir_rand6)
#
if not os.path.exists(out_dir_rand8):
    os.makedirs(out_dir_rand8)    

In [14]:
fname_node_label_train = "label.dat"
fname_node_label_test = "label.dat.test"
fname_link = "link.dat" 
fname_link_test = "link.dat.test"
fname_node = "node.dat"
#fname_all_data_dict = data_dir + "pubmed_heuristic_data_dict_v"+version_no+"_p2.pkl"
fname_all_data_dict = data_dir + "pubmed_heuristic_data_dict_v"+version_no+"_p2.pkl.pbz2"

In [15]:
dname+fname_node_label_train

'./../../data_hin/PubMed_orig/label.dat'

In [16]:
df_node_label_train = pd.read_csv(open(dname +fname_node_label_train),sep="\s",header=None)
df_node_label_train.columns = ["node_id", "node_name", "node_type", "node_label"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [17]:
df_node_label_test = pd.read_csv(open(dname +fname_node_label_test),sep="\s",header=None)
df_node_label_test.columns = ["node_id", "node_name", "node_type", "node_label"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [18]:
df_link = pd.read_csv(open(dname+fname_link),sep="\s",header=None)
df_link.columns = ["start_node_id", "end_node_id", "link_type", "link_weight"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [19]:
df_link_full = pd.read_csv(open(dname+fname_link),sep="\s",header=None)
df_link_full.columns = ["start_node_id", "end_node_id", "link_type", "link_weight"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [20]:
df_link_test = pd.read_csv(open(dname+fname_link_test),sep="\s",header=None)
df_link_test.columns = ["start_node_id", "end_node_id", "link_status"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [21]:
df_node = pd.read_csv(open(dname+fname_node),sep="\s",header=None)
df_node.columns = ["node_id", "node_name", "node_type","node_attrs"]

/home/ragu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [22]:
#all_data_dict = pkl.load(open(fname_all_data_dict,"rb"))
all_data_dict = data_dict = decompress_pickle(fname_all_data_dict)

In [23]:
list_x_id_new = list(all_data_dict["matrices"].keys())
list_e_id_new = list(all_data_dict["metadata"]["dict_e_size"].keys())

In [24]:
dict_map_xid = {'X11': "X0",
 'X12': "X1",
 'X22': "X2",
 'X31': "X3",
 'X32': "X4",
 'X33': "X5",
 'X34': "X6",
 'X41': "X7",
 'X42': "X8",
 'X44': "X9"}

In [25]:
dict_map_eid = {
    "e1":"e0",
    "e2":"e1",
    "e3":"e2",
    "e4":"e3"
}

In [26]:
list_x_id = []
for x_id in list_x_id_new:
    list_x_id.append(dict_map_xid[x_id])
#
list_e_id = []
for e_id in list_e_id_new:
    list_e_id.append(dict_map_eid[e_id])

In [27]:
#

In [28]:
# fname_prefix = "X"
# fname_suffix = ".npy"
# fname = dname_mat 

In [29]:
fname_cc_info = out_dir + "dict_path_cc_info.pkl" #<< from the CC selection
#fname_cc_select = dname_mat + + "dict_path_cc_select_info.pkl" #<< from the CC selection
fname_cc_edges_select =  out_dir + "dict_path_cc_select_info_edges.pkl" #<< from the CC selection
##fname_map_matidx_nodeid = dname_mat +"id_idx.csv"

In [30]:
num_matrices = 10
print("Loading the matrices: ")
data_dict = {}
for x_id in all_data_dict["matrices"].keys():
    new_x_id = dict_map_xid[x_id]
    data_dict[new_x_id] = all_data_dict["matrices"][x_id]
    print("x_id: ",new_x_id," X.shape: ",data_dict[new_x_id].shape)
print("#")

Loading the matrices: 
x_id:  X4  X.shape:  (5660, 4405)
x_id:  X5  X.shape:  (5660, 5660)
x_id:  X3  X.shape:  (5660, 2634)
x_id:  X7  X.shape:  (608, 2634)
x_id:  X6  X.shape:  (5660, 608)
x_id:  X8  X.shape:  (608, 4405)
x_id:  X9  X.shape:  (608, 608)
x_id:  X0  X.shape:  (2634, 2634)
x_id:  X1  X.shape:  (2634, 4405)
x_id:  X2  X.shape:  (4405, 4405)
#


In [31]:
data_dict.keys()

dict_keys(['X4', 'X5', 'X3', 'X7', 'X6', 'X8', 'X9', 'X0', 'X1', 'X2'])

In [32]:
for x_id in data_dict:
    if x_id.startswith("X"):
        temp_x = data_dict[x_id]
        print("x_id: ",x_id,", %nz: ",100 - ((np.sum(temp_x) / np.prod(temp_x.shape)) * 100))

x_id:  X4 , %nz:  99.79048062152309
x_id:  X5 , %nz:  99.96037533244265
x_id:  X3 , %nz:  99.86735030626947
x_id:  X7 , %nz:  99.90621128561723
x_id:  X6 , %nz:  99.93110121815138
x_id:  X8 , %nz:  99.84964006213035
x_id:  X9 , %nz:  99.96050467451524
x_id:  X0 , %nz:  99.87630362602472
x_id:  X1 , %nz:  99.6920390561909
x_id:  X2 , %nz:  99.86215231118285


In [33]:
all_data_dict["metadata"].keys()

dict_keys(['dict_e_id_to_idx_map', 'dict_e_idx_to_id_map', 'x_meta', 'dict_e_size'])

In [34]:
all_data_dict["metadata"]["dict_e_id_to_idx_map"].keys()

dict_keys(['e4', 'e1', 'e3', 'e2'])

In [35]:
def mode_xid(dict_with_key_xid):
    dict_temp = {}
    for xid in dict_with_key_xid.keys():
        temp_xid = dict_map_xid[xid].replace("X","")
        dict_temp[temp_xid] = dict_with_key_xid[xid]
    return dict_temp

# def mode_eid_xid(dict_with_key_eid_xid):
#     dict_temp = {}
#     for eid_xid in dict_with_key_eid_xid.keys():
#         eid = eid_xid.split("_")[0]
#         xid = eid_xid.split("_")[1]
#         temp_xid = dict_map_xid[xid]
#         temp_eid = dict_map_eid[eid]
#         temp_eid_xid = temp_eid+"_"+temp_xid
#         dict_temp[temp_eid_xid] = dict_with_key_eid_xid[eid_xid]
#     return dict_temp

def mode_eid(dict_with_key_eid):
    dict_temp = {}
    for eid in dict_with_key_eid.keys():
        temp_eid = int(dict_map_eid[eid].replace("e",""))
        dict_temp[temp_eid] = dict_with_key_eid[eid]
    return dict_temp

In [36]:
dict_map_nodeid_matidx = all_data_dict["metadata"]["dict_e_id_to_idx_map"]
dict_map_matidx_nodeid = all_data_dict["metadata"]["dict_e_idx_to_id_map"]

In [37]:
dict_map_nodeid_matidx.keys()

dict_keys(['e4', 'e1', 'e3', 'e2'])

In [38]:
dict_map_matidx_nodeid.keys()

dict_keys(['e4', 'e1', 'e3', 'e2'])

In [39]:
dict_map_nodeid_matidx = mode_eid(dict_map_nodeid_matidx)
dict_map_matidx_nodeid = mode_eid(dict_map_matidx_nodeid)

In [40]:
dict_map_nodeid_matidx.keys()

dict_keys([3, 0, 2, 1])

In [41]:
dict_map_matidx_nodeid.keys()

dict_keys([3, 0, 2, 1])

In [42]:
# df_map_matidx_nodeid = pd.read_csv(fname_map_matidx_nodeid, header=None)
# df_map_matidx_nodeid.columns = ["node_id","idx"]

In [43]:
dict_cc_info = pkl.load(open(fname_cc_info,"rb"))
#dict_cc_select = pkl.load(open(fname_cc_select,"rb"))
dict_cc_edges_select = pkl.load(open(fname_cc_edges_select,"rb"))

In [44]:
# df_map_matidx_nodeid[:3]

In [45]:
dict_cc_info

{(2, 0): {'chain_template_id': 0,
  'cur_chain_template_e': {'p1': ['e2', 'e1'], 'p2': ['e2', 'e0', 'e1']},
  'cur_chain_template_x': {'p1': 'X4', 'p2': ['X3', 'X1']},
  'path_id': 2},
 (2, 1): {'chain_template_id': 1,
  'cur_chain_template_e': {'p1': ['e2', 'e1'], 'p2': ['e2', 'e3', 'e0', 'e1']},
  'cur_chain_template_x': {'p1': 'X4', 'p2': ['X6', 'X7', 'X1']},
  'path_id': 2},
 (2, 2): {'chain_template_id': 2,
  'cur_chain_template_e': {'p1': ['e2', 'e1'], 'p2': ['e2', 'e3', 'e1']},
  'cur_chain_template_x': {'p1': 'X4', 'p2': ['X6', 'X8']},
  'path_id': 2},
 (3, 0): {'chain_template_id': 0,
  'cur_chain_template_e': {'p1': ['e3', 'e1'], 'p2': ['e3', 'e0', 'e1']},
  'cur_chain_template_x': {'p1': 'X8', 'p2': ['X7', 'X1']},
  'path_id': 3}}

In [46]:
df_link_copy1 = df_link.copy() #max
df_link_copy2 = None
#df_link_copy2 = df_link.copy() #min
#
# df_link_copy3 = df_link.copy() #rand1-max
# df_link_copy4 = df_link.copy() #rand2 -const
# df_link_copy5 = df_link.copy() #rand3-min
#
df_link_copy6 = df_link.copy() #rand7-max
df_link_copy7 = None
#df_link_copy7 = df_link.copy() #rand5-min
#df_link_copy8 = df_link.copy() #rand6-const

In [47]:
# #update df_map_matidx_nodeid with node_type
# dict_temp_nt = {}
# for idx,row in df_map_matidx_nodeid.iterrows():
#     cur_id = row["node_id"]
#     cur_idx = row["idx"]
#     df_temp = df_node[df_node["node_id"].isin([cur_id])]
#     assert df_temp.shape[0] == 1
#     cur_node_type = df_temp["node_type"].item()
#     dict_temp_nt[idx] = {
#         "node_id":cur_id,
#         "idx":cur_idx,
#         "node_type":cur_node_type
#     }

In [48]:
# df_map_matidx_nodeid_new = pd.DataFrame(dict_temp_nt).T

In [49]:
# df_map_matidx_nodeid_new.shape

In [50]:
# df_map_matidx_nodeid_new.head()

In [51]:
# df_map_matidx_nodeid_new.tail()

In [52]:
# df_map_matidx_nodeid = df_map_matidx_nodeid_new[["node_id", "node_type","idx"]]

In [53]:
# df_map_matidx_nodeid[:5]

In [54]:
dict_subset_start_count_orig = df_link.groupby("start_node_id").agg(["count"])["end_node_id"].to_dict()["count"]
dict_subset_end_count_orig = df_link.groupby("end_node_id").agg(["count"])["start_node_id"].to_dict()["count"]
print("len(dict_subset_start_count_orig): ",len(dict_subset_start_count_orig))
print("len(dict_subset_end_count_orig): ",len(dict_subset_end_count_orig))        

len(dict_subset_start_count_orig):  11589
len(dict_subset_end_count_orig):  10168


In [55]:
assert df_node.shape[0] == np.unique(list(dict_subset_start_count_orig.keys()) + list(dict_subset_end_count_orig.keys())).shape[0]

In [56]:
############################################## max #####################################

In [57]:
dict_subset_start_count = dict_subset_start_count_orig.copy()
dict_subset_end_count = dict_subset_end_count_orig.copy()

In [58]:
dict_count_max = {}
#
df_link = df_link_copy1
min_or_max = "max"
print("##################")
print("min_or_max: ",min_or_max)
print("BEFORE df_link.shape: ",df_link.shape)
print("##################")
count_path_all = 0
count_path_all_in = 0
prev_count = df_link.shape[0]
for path_id in dict_cc_edges_select:
    dict_count_max[path_id] = {}
    print("path_id: ",path_id)
    print("###")
    cur_path_dict = dict_cc_edges_select[path_id][min_or_max]
    #
    #list_edges_to_filter = []
    #list_nodes = []
    list_df = []
    count_all = 0
    count_all_in = 0
    for x_id in cur_path_dict:
        print("x_id: ",x_id)
        cur_X = data_dict[x_id]
        cur_X_dict = cur_path_dict[x_id]
        col_e_id = cur_X_dict["col_e_id"]
        col_node_type = int(col_e_id.replace("e",""))
        row_e_id = cur_X_dict["row_e_id"]
        row_node_type = int(row_e_id.replace("e",""))
        col_idx_list = list(cur_X_dict["col_idx_list"])
        row_idx_list = list(cur_X_dict["row_idx_list"])
        #obtain the corresponding row and column node ids
        #df_row_nodes = df_map_matidx_nodeid[df_map_matidx_nodeid["node_type"].isin([row_node_type])]
        #list_row_node_id_list = df_row_nodes[df_row_nodes["idx"].isin(row_idx_list)]["node_id"].unique().tolist()
        list_row_node_id_list = []
        for temp_idx in row_idx_list:
            list_row_node_id_list.append(dict_map_matidx_nodeid[row_node_type][temp_idx])
        #df_col_nodes = df_map_matidx_nodeid[df_map_matidx_nodeid["node_type"].isin([col_node_type])]
        #list_col_node_id_list = df_col_nodes[df_col_nodes["idx"].isin(col_idx_list)]["node_id"].unique().tolist()
        list_col_node_id_list = []
        for temp_idx in col_idx_list:
            list_col_node_id_list.append(dict_map_matidx_nodeid[col_node_type][temp_idx])
        #
        df_link_subset = df_link[df_link["start_node_id"].isin(list_row_node_id_list)\
                                 & df_link["end_node_id"].isin(list_col_node_id_list)]
        print("df_link_subset.shape: ",df_link_subset.shape)
        #
        count = 0
        for idx, row in df_link_subset.iterrows():
            count_all_in+=1
            count_path_all_in+=1
            row_id = row["start_node_id"]
            col_id = row["end_node_id"]
            if dict_subset_start_count[row_id] > 1 and dict_subset_end_count[col_id] > 1:
                list_df.append(pd.DataFrame(row).T)
                dict_subset_start_count[row_id] = dict_subset_start_count[row_id] - 1
                dict_subset_end_count[col_id] = dict_subset_end_count[col_id] - 1
                count+=1
                count_all+=1
                count_path_all+=1
        print("count: ",count)
        print("#")
        dict_count_max[path_id][x_id] = count
    print("######")
    print("before df_link.shape: ",df_link.shape)
    if len(list_df) > 0:
        df_link = pd.concat([df_link,pd.concat(list_df)]).drop_duplicates(keep=False)   
    print("count_all: ",count_all)
    print("count_all_in: ",count_all_in)
    print("after df_link.shape: ",df_link.shape)
    print("#######")
print("##################")
next_count = df_link.shape[0]
print("After df_link.shape: ",df_link.shape)
print("count_path_all_in: ",count_path_all_in)
print("count_path_all: ",count_path_all)
print("#")
print("Total removed: ",prev_count - next_count)
print("##################")
num_edges_removed_max = count_path_all

##################
min_or_max:  max
BEFORE df_link.shape:  (40189, 4)
##################
path_id:  (2, 0)
###
x_id:  X4
df_link_subset.shape:  (2, 4)
count:  1
#
x_id:  X3
df_link_subset.shape:  (308, 4)
count:  263
#
x_id:  X1
df_link_subset.shape:  (7, 4)
count:  7
#
######
before df_link.shape:  (40189, 4)
count_all:  271
count_all_in:  317
after df_link.shape:  (39918, 4)
#######
path_id:  (2, 1)
###
x_id:  X4
df_link_subset.shape:  (1, 4)
count:  0
#
x_id:  X6
df_link_subset.shape:  (50, 4)
count:  23
#
x_id:  X7
df_link_subset.shape:  (2, 4)
count:  1
#
x_id:  X1
df_link_subset.shape:  (0, 4)
count:  0
#
######
before df_link.shape:  (39918, 4)
count_all:  24
count_all_in:  53
after df_link.shape:  (39894, 4)
#######
path_id:  (2, 2)
###
x_id:  X4
df_link_subset.shape:  (1, 4)
count:  0
#
x_id:  X6
df_link_subset.shape:  (27, 4)
count:  0
#
x_id:  X8
df_link_subset.shape:  (0, 4)
count:  0
#
######
before df_link.shape:  (39894, 4)
count_all:  0
count_all_in:  28
after df_link.sh

In [59]:
dict_count_max

{(2, 0): {'X1': 7, 'X3': 263, 'X4': 1},
 (2, 1): {'X1': 0, 'X4': 0, 'X6': 23, 'X7': 1},
 (2, 2): {'X4': 0, 'X6': 0, 'X8': 0},
 (3, 0): {'X1': 2, 'X7': 6, 'X8': 0}}

In [60]:
#df_link.to_csv(out_dir_min_max+fname_link,index=False, sep="\t",header=False)

In [61]:
############################################## min #####################################

In [62]:
#dict_subset_start_count = dict_subset_start_count_orig.copy()
#dict_subset_end_count = dict_subset_end_count_orig.copy()

In [63]:
dict_count_min = {}
#
#df_link = df_link_copy2 #continue to use the same df_link with max links removed
min_or_max = "min"
print("##################")
print("min_or_max: ",min_or_max)
print("BEFORE df_link.shape: ",df_link.shape)
print("##################")
count_path_all = 0
count_path_all_in = 0
prev_count = df_link.shape[0]
for path_id in dict_cc_edges_select:
    print("path_id: ",path_id)
    print("###")
    dict_count_min[path_id] = {}
    cur_path_dict = dict_cc_edges_select[path_id][min_or_max]
    #
    #list_edges_to_filter = []
    #list_nodes = []
    list_df = []
    count_all = 0
    count_all_in = 0
    for x_id in cur_path_dict:
        print("x_id: ",x_id)
        cur_X = data_dict[x_id]
        cur_X_dict = cur_path_dict[x_id]
        col_e_id = cur_X_dict["col_e_id"]
        col_node_type = int(col_e_id.replace("e",""))
        row_e_id = cur_X_dict["row_e_id"]
        row_node_type = int(row_e_id.replace("e",""))
        col_idx_list = list(cur_X_dict["col_idx_list"])
        row_idx_list = list(cur_X_dict["row_idx_list"])
        #obtain the corresponding row and column node ids
#         df_row_nodes = df_map_matidx_nodeid[df_map_matidx_nodeid["node_type"].isin([row_node_type])]
#         list_row_node_id_list = df_row_nodes[df_row_nodes["idx"].isin(row_idx_list)]["node_id"].unique().tolist()
#         df_col_nodes = df_map_matidx_nodeid[df_map_matidx_nodeid["node_type"].isin([col_node_type])]
#         list_col_node_id_list = df_col_nodes[df_col_nodes["idx"].isin(col_idx_list)]["node_id"].unique().tolist()
        list_row_node_id_list = []
        for temp_idx in row_idx_list:
            list_row_node_id_list.append(dict_map_matidx_nodeid[row_node_type][temp_idx])
        list_col_node_id_list = []
        for temp_idx in col_idx_list:
            list_col_node_id_list.append(dict_map_matidx_nodeid[col_node_type][temp_idx])
        #
        df_link_subset = df_link[df_link["start_node_id"].isin(list_row_node_id_list)\
                                 & df_link["end_node_id"].isin(list_col_node_id_list)]
        print("df_link_subset.shape: ",df_link_subset.shape)
        #
        count = 0
        for idx, row in df_link_subset.iterrows():
            count_all_in+=1
            count_path_all_in+=1
            row_id = row["start_node_id"]
            col_id = row["end_node_id"]
            if dict_subset_start_count[row_id] > 1 and dict_subset_end_count[col_id] > 1:
                list_df.append(pd.DataFrame(row).T)
                dict_subset_start_count[row_id] = dict_subset_start_count[row_id] - 1
                dict_subset_end_count[col_id] = dict_subset_end_count[col_id] - 1
                count+=1
                count_all+=1
                count_path_all+=1
        print("count: ",count)
        print("#")
        dict_count_min[path_id][x_id] = count
    print("######")
    print("before df_link.shape: ",df_link.shape)
    if len(list_df) > 0:
        df_link = pd.concat([df_link,pd.concat(list_df)]).drop_duplicates(keep=False)   
    print("count_all: ",count_all)
    print("count_all_in: ",count_all_in)
    print("after df_link.shape: ",df_link.shape)
    print("#######")
print("##################")
next_count = df_link.shape[0]
print("After df_link.shape: ",df_link.shape)
print("count_path_all_in: ",count_path_all_in)
print("count_path_all: ",count_path_all)
print("#")
print("Total removed: ",prev_count - next_count)
print("##################")
num_edges_removed_min = count_path_all

##################
min_or_max:  min
BEFORE df_link.shape:  (39886, 4)
##################
path_id:  (2, 0)
###
x_id:  X4
df_link_subset.shape:  (330, 4)
count:  247
#
x_id:  X3
df_link_subset.shape:  (45, 4)
count:  0
#
x_id:  X1
df_link_subset.shape:  (0, 4)
count:  0
#
######
before df_link.shape:  (39886, 4)
count_all:  247
count_all_in:  375
after df_link.shape:  (39639, 4)
#######
path_id:  (2, 1)
###
x_id:  X4
df_link_subset.shape:  (83, 4)
count:  0
#
x_id:  X6
df_link_subset.shape:  (27, 4)
count:  0
#
x_id:  X7
df_link_subset.shape:  (1, 4)
count:  0
#
x_id:  X1
df_link_subset.shape:  (0, 4)
count:  0
#
######
before df_link.shape:  (39639, 4)
count_all:  0
count_all_in:  111
after df_link.shape:  (39639, 4)
#######
path_id:  (2, 2)
###
x_id:  X4
df_link_subset.shape:  (83, 4)
count:  0
#
x_id:  X6
df_link_subset.shape:  (27, 4)
count:  0
#
x_id:  X8
df_link_subset.shape:  (0, 4)
count:  0
#
######
before df_link.shape:  (39639, 4)
count_all:  0
count_all_in:  110
after df_link

In [64]:
39161 - 40189

-1028

In [65]:
dict_count_min

{(2, 0): {'X1': 0, 'X3': 0, 'X4': 247},
 (2, 1): {'X1': 0, 'X4': 0, 'X6': 0, 'X7': 0},
 (2, 2): {'X4': 0, 'X6': 0, 'X8': 0},
 (3, 0): {'X1': 0, 'X7': 0, 'X8': 15}}

In [66]:
dict_count_max

{(2, 0): {'X1': 7, 'X3': 263, 'X4': 1},
 (2, 1): {'X1': 0, 'X4': 0, 'X6': 23, 'X7': 1},
 (2, 2): {'X4': 0, 'X6': 0, 'X8': 0},
 (3, 0): {'X1': 2, 'X7': 6, 'X8': 0}}

In [67]:
dict_link_type_num_removed1 = {}
for link_type in np.arange(num_matrices):
    df_link_type_subset_rand = df_link[df_link["link_type"].isin([link_type])]
    df_link_type_subset_full = df_link_full[df_link_full["link_type"].isin([link_type])]
    num_removed = df_link_type_subset_full.shape[0] - df_link_type_subset_rand.shape[0]
    print("link_type: ",link_type," #nodes removed: ",num_removed)
    print("df_link_type_subset_full.shape[0]: ",df_link_type_subset_full.shape[0])
    print("df_link_type_subset_rand.shape[0]: ",df_link_type_subset_rand.shape[0])
    print("#")
    dict_link_type_num_removed1[link_type] = num_removed

link_type:  0  #nodes removed:  0
df_link_type_subset_full.shape[0]:  5142
df_link_type_subset_rand.shape[0]:  5142
#
link_type:  1  #nodes removed:  9
df_link_type_subset_full.shape[0]:  4851
df_link_type_subset_rand.shape[0]:  4842
#
link_type:  2  #nodes removed:  3
df_link_type_subset_full.shape[0]:  5732
df_link_type_subset_rand.shape[0]:  5729
#
link_type:  3  #nodes removed:  276
df_link_type_subset_full.shape[0]:  5727
df_link_type_subset_rand.shape[0]:  5451
#
link_type:  4  #nodes removed:  245
df_link_type_subset_full.shape[0]:  7494
df_link_type_subset_rand.shape[0]:  7249
#
link_type:  5  #nodes removed:  22
df_link_type_subset_full.shape[0]:  8747
df_link_type_subset_rand.shape[0]:  8725
#
link_type:  6  #nodes removed:  1
df_link_type_subset_full.shape[0]:  910
df_link_type_subset_rand.shape[0]:  909
#
link_type:  7  #nodes removed:  2
df_link_type_subset_full.shape[0]:  639
df_link_type_subset_rand.shape[0]:  637
#
link_type:  8  #nodes removed:  7
df_link_type_subset_f

In [68]:
dict_link_type_num_removed1

{0: 0, 1: 9, 2: 3, 3: 276, 4: 245, 5: 22, 6: 1, 7: 2, 8: 7, 9: 0}

In [69]:
df_link.to_csv(out_dir_min_max+fname_link,index=False, sep="\t",header=False)

In [70]:
################################# rand 6 uniform max #############################

In [71]:
def get_pruned_df_link_uniform(df_link, dict_count_max, \
                        dict_subset_start_count, dict_subset_end_count):

    dict_count_rand4_max = {}
    num_edges_to_hide_threshold = 0
    num_sampling_try_threshold = 100
    print("##################")
    print("BEFORE df_link.shape: ",df_link.shape)
    print("##################")
    count_path_all = 0
    count_path_all_in = 0
    prev_count = df_link.shape[0]
    #
    tot_num_edges_hidden_max = 200
#     for path_id in dict_count_max:
#         tot_num_edges_hidden_max += np.sum(list(dict_count_max[path_id].values()))
    print("tot_num_edges_hidden_max: ",tot_num_edges_hidden_max)        
    print("#")
    #
    num_matrices = 10
    #
    #>>> num_edges_to_hide = int(tot_num_edges_hidden_max/num_matrices) #per matrix
    #print("num_edges_to_hide, per mat: ",num_edges_to_hide)
    #
#     list_rand_matrices = []
#     list_rand_matrices.append(np.random.choice(np.arange(num_matrices)))
#     list_rand_matrices.append(np.random.choice(np.arange(num_matrices)))
#     list_rand_matrices.append(np.random.choice(np.arange(num_matrices)))
    for link_type in np.arange(num_matrices):
    #for link_type in list_rand_matrices:
        num_edges_to_hide_temp = int(tot_num_edges_hidden_max /np.random.choice(np.arange(8,10)+1))
        num_edges_to_hide = np.random.choice([0,0,0,0,num_edges_to_hide_temp])
        print("num_edges_to_hide, per mat: ",num_edges_to_hide)
        list_df = []
        df_link_type_subset = df_link[df_link["link_type"].isin([link_type])]
        #
        count = 0
        count_in = 0
        num_sampling_try = 0   
        while count < num_edges_to_hide:
            cur_size = df_link_type_subset.shape[0]
            if 3*num_edges_to_hide > cur_size:
                df_link_subset = df_link_type_subset.sample(n=cur_size,random_state=1)
            else:
                df_link_subset = df_link_type_subset.sample(n=3*num_edges_to_hide,random_state=1)
            num_sampling_try+=1
            print("num_sampling_try: ",num_sampling_try)
            for idx, row in df_link_subset.iterrows():
                count_in+=1
                count_path_all_in+=1
                row_id = row["start_node_id"]
                col_id = row["end_node_id"]
                if dict_subset_start_count[row_id] > 1 and dict_subset_end_count[col_id] > 1:
                    list_df.append(pd.DataFrame(row).T)
                    dict_subset_start_count[row_id] = dict_subset_start_count[row_id] - 1
                    dict_subset_end_count[col_id] = dict_subset_end_count[col_id] - 1
                    count+=1
                    count_path_all+=1           
                if not count < num_edges_to_hide:
                    break
            if num_sampling_try > num_sampling_try_threshold:
                break
        #
        if len(list_df) > 0:
            df_link_subset = pd.concat(list_df)
            #assert df_link_subset.shape[0] == num_edges_to_hide,"df_link_subset.shape[0]: "+str(df_link_subset.shape[0])+", num_edges_to_hide: "+str(num_edges_to_hide)
            #
            print("#")
            print("df_link_subset.shape: ",df_link_subset.shape)
            print("df_link_subset.group: ")
            pp.pprint(df_link_subset.groupby("link_type").agg(["count"])["start_node_id"])
            print("#")
            print("before df_link.shape: ",df_link.shape)
            bef_count = df_link.shape[0]
            df_link = pd.concat([df_link,df_link_subset]).drop_duplicates(keep=False)
            aft_count = df_link.shape[0]
            print("count: ",count)
            print("count_in: ",count_in)
            print("after df_link.shape: ",df_link.shape)
            print("#")   
            dict_count_rand4_max[link_type] = bef_count - aft_count
            assert dict_count_rand4_max[link_type] <= num_edges_to_hide
        else:
            print("Skipping hiding - could not find samples that meet our criteria. ")
    print("##################")
    next_count = df_link.shape[0]
    print("AFTER df_link.shape: ",df_link.shape)
    print("count_path_all_in: ",count_path_all_in)
    print("count_path_all: ",count_path_all)
    print("#")
    print("Total removed: ",prev_count - next_count)
    print("##################") 
    return df_link, dict_count_rand4_max

In [72]:
df_link = df_link_copy6

In [73]:
dict_subset_start_count = dict_subset_start_count_orig.copy()
dict_subset_end_count = dict_subset_end_count_orig.copy()

In [74]:
dict_count_max

{(2, 0): {'X1': 7, 'X3': 263, 'X4': 1},
 (2, 1): {'X1': 0, 'X4': 0, 'X6': 23, 'X7': 1},
 (2, 2): {'X4': 0, 'X6': 0, 'X8': 0},
 (3, 0): {'X1': 2, 'X7': 6, 'X8': 0}}

In [75]:
#repeatedly run the below until the required number of nodes are removed
df_link, dict_count_rand6_max_uniform = get_pruned_df_link_uniform(df_link, dict_count_max, \
                        dict_subset_start_count, dict_subset_end_count)

##################
BEFORE df_link.shape:  (40189, 4)
##################
tot_num_edges_hidden_max:  200
#
num_edges_to_hide, per mat:  20
num_sampling_try:  1
#
df_link_subset.shape:  (20, 4)
df_link_subset.group: 
           count
link_type       
0             20
#
before df_link.shape:  (40189, 4)
count:  20
count_in:  24
after df_link.shape:  (40169, 4)
#
num_edges_to_hide, per mat:  0
Skipping hiding - could not find samples that meet our criteria. 
num_edges_to_hide, per mat:  0
Skipping hiding - could not find samples that meet our criteria. 
num_edges_to_hide, per mat:  20
num_sampling_try:  1
#
df_link_subset.shape:  (20, 4)
df_link_subset.group: 
           count
link_type       
3             20
#
before df_link.shape:  (40169, 4)
count:  20
count_in:  22
after df_link.shape:  (40149, 4)
#
num_edges_to_hide, per mat:  0
Skipping hiding - could not find samples that meet our criteria. 
num_edges_to_hide, per mat:  0
Skipping hiding - could not find samples that meet our criter

In [76]:
294 + 200 + 422 + 20 + 33 + 82

1051

In [77]:
39151 - 40189

-1038

In [78]:
df_link.to_csv(out_dir_rand8+fname_link,index=False, sep="\t",header=False)

In [79]:
#how many of each node_type were removed randomly?

In [80]:
df_link_full.shape

(40189, 4)

In [81]:
df_link.shape

(40149, 4)

In [82]:
dict_link_type_num_removed = {}
for link_type in np.arange(num_matrices):
    df_link_type_subset_rand = df_link[df_link["link_type"].isin([link_type])]
    df_link_type_subset_full = df_link_full[df_link_full["link_type"].isin([link_type])]
    num_removed = df_link_type_subset_full.shape[0] - df_link_type_subset_rand.shape[0]
    print("link_type: ",link_type," #nodes removed: ",num_removed)
    print("df_link_type_subset_full.shape[0]: ",df_link_type_subset_full.shape[0])
    print("df_link_type_subset_rand.shape[0]: ",df_link_type_subset_rand.shape[0])
    print("#")
    dict_link_type_num_removed[link_type] = num_removed

link_type:  0  #nodes removed:  20
df_link_type_subset_full.shape[0]:  5142
df_link_type_subset_rand.shape[0]:  5122
#
link_type:  1  #nodes removed:  0
df_link_type_subset_full.shape[0]:  4851
df_link_type_subset_rand.shape[0]:  4851
#
link_type:  2  #nodes removed:  0
df_link_type_subset_full.shape[0]:  5732
df_link_type_subset_rand.shape[0]:  5732
#
link_type:  3  #nodes removed:  20
df_link_type_subset_full.shape[0]:  5727
df_link_type_subset_rand.shape[0]:  5707
#
link_type:  4  #nodes removed:  0
df_link_type_subset_full.shape[0]:  7494
df_link_type_subset_rand.shape[0]:  7494
#
link_type:  5  #nodes removed:  0
df_link_type_subset_full.shape[0]:  8747
df_link_type_subset_rand.shape[0]:  8747
#
link_type:  6  #nodes removed:  0
df_link_type_subset_full.shape[0]:  910
df_link_type_subset_rand.shape[0]:  910
#
link_type:  7  #nodes removed:  0
df_link_type_subset_full.shape[0]:  639
df_link_type_subset_rand.shape[0]:  639
#
link_type:  8  #nodes removed:  0
df_link_type_subset_full

In [83]:
pp.pprint(dict_link_type_num_removed)

{0: 20, 1: 0, 2: 0, 3: 20, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}


In [84]:
dict_link_type_num_removed1

{0: 0, 1: 9, 2: 3, 3: 276, 4: 245, 5: 22, 6: 1, 7: 2, 8: 7, 9: 0}